In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf

# Function for data augmentation
def apply_augmentation(image, mask):
    # Random horizontal flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    # Random vertical flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 0)
        mask = cv2.flip(mask, 0)

    # Random rotation
    if np.random.rand() > 0.5:
        angle = np.random.uniform(-45, 45)  # Adjust rotation angle 
        rows, cols = image.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_NEAREST)
        mask = cv2.warpAffine(mask, M, (cols, rows), flags=cv2.INTER_NEAREST)

    # Random zoom
    if np.random.rand() > 0.5:
        zoom_factor = np.random.uniform(0.8, 1.2)
        rows, cols = image.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), 0, zoom_factor)
        image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_NEAREST)
        mask = cv2.warpAffine(mask, M, (cols, rows), flags=cv2.INTER_NEAREST)

    # Random brightness adjustment
    if np.random.rand() > 0.5:
        brightness_factor = np.random.uniform(0.5, 2.0)
        image = np.clip(image * brightness_factor, 0, 255)

    return image, mask

# Function for loading and preprocessing images and masks
def load_and_preprocess_data(image_path, mask_path, augment=False, resize_shape=(256, 256)):
    # Load fluorescence microscopy image and mask
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Resize
    image = cv2.resize(image, dsize=resize_shape, interpolation=cv2.INTER_AREA)
    mask = cv2.resize(mask, dsize=resize_shape, interpolation=cv2.INTER_AREA)

    # Normalize to [0, 1]
    image = image / 255.0
    mask = mask / 255.0

    # Data augmentation
    if augment:
        image, mask = apply_augmentation(image, mask)

    return image, mask

# Function to process data for a given directory
def process_data(directory, mask_directory, augment=False):
    data = []
    mask_data = []

    images_path = sorted(os.listdir(directory))
    for image_name in images_path:
        image_path = os.path.join(directory, image_name)
        mask_path = os.path.join(mask_directory, image_name)

        image, mask = load_and_preprocess_data(image_path, mask_path, augment=augment)
        
        data.append(np.array(image))
        mask_data.append(np.array(mask))

    data = np.expand_dims(np.array(data), axis=-1)  # Add channel dimension
    mask_data = np.expand_dims(np.array(mask_data), axis=-1)  # Add channel dimension

    return data, mask_data

# Define paths to your datasets
train_data_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/train/images/'
train_mask_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/train/masks/'

val_data_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/val/images/'
val_mask_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/val/masks/'

test_data_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/test/images/'
test_mask_dir = 'Fluorescence Microscopy Images v2/cbmi_MITO/test/masks/'

# Process training data with augmentation
Train_data, Tmask_data = process_data(train_data_dir, train_mask_dir, augment=True)

# Process validation data without augmentation
Val_data, Vmask_data = process_data(val_data_dir, val_mask_dir, augment=False)

# Process test data without augmentation
Test_data, Smask_data = process_data(test_data_dir, test_mask_dir, augment=False)


In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dropout

#U-net model

kernel_initializer = 'he_uniform'

inputs = Input((Train_data.shape[1], Train_data.shape[2], Train_data.shape[3]))


#Contraction path
s = inputs

#Contraction path
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)

c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)

c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)

c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)

#Expansive path
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
c8 = Dropout(0.1)(c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = concatenate([u9, c1])
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
c9 = Dropout(0.1)(c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

num_classes = 2  
iou_metric = tf.keras.metrics.MeanIoU(num_classes=num_classes)

UNet = Model(inputs=[inputs], outputs=[outputs])
#compile model outside of this function to make it flexible.
UNet.compile(optimizer='adam', loss='BinaryCrossentropy',metrics=['accuracy', tf.keras.metrics.BinaryIoU(),iou_metric])
UNet.summary()

In [ ]:
history_unet = UNet.fit(x = Train_data,
                        y = Tmask_data,
                        batch_size = 16,
                        validation_data = (Val_data, Vmask_data),
                        verbose = 1, 
                        epochs = 60)